In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Collision resolution: Chaining

Our current collision resolution strategy simply rejects key-value pairs where the key maps onto an already occupied hash table location. An obvious alternative strategy is to overwrite the existing key-value pair. Both strategies result in adding a key-value pair in $O(1)$ time but they have obvious significant drawbacks.

## Chaining

In *chaining* or *separate chaining* each bucket holds a reference to some kind of container. When a collision occurs with a key $k$, the key-value pair is simply added to the container if no pair in the container has the key $k$. If a pair in the container has the key $k$ then the value of the pair is replaced with the new value.

### Chaining with linked lists

In a hash table that uses chaining with linked lists each bucket contains a reference to a linked list; in other words, the array $t$ becomes an array of linked lists. Instead of using a full featured linked list, we can use a linked-node structure identical to the one that we used when implementing linked stacks. The Java standard library class `java.util.Hashtable` is an example of an implementation that uses chaining with a linked-node structure.

The `contains`, `get`, `put` and `remove` operations all involve searching a linked list for a key when a collision occurs. In the worst-case, all $n$ key-value pairs in the hash table can be located in a single linked list; therefore, the worst-case complexity of these operations is in $O(n)$.

To compute the average-case complexity, we need to assume something about how the keys are distributed among the $m$ buckets of the hash table. The *simple uniform hashing* assumption assumes that keys are distributed with equal probability for every bucket.

The *load factor* $\alpha$ is defined as the ratio $\alpha = n / m$ where $n$ is the number of key-value pairs in the hash table and $m$ is the number of buckets.

On average, the length of a linked list in a hash table using chaining with linked lists is equal to the load factor under the simple uniform hashing assumption.

**Claim** Under the simple uniform hashing assumption the complexity of searching for a key-value pair that *is not* in the hash table is in $\Theta(1 + \alpha)$ on average.

**Proof** The complexity of computing the hash function for a key $k$ is in $\Theta(1)$. If a key-value pair is not in the hash table then we need to examine every element of a linked list testing the key of each element for equality to $k$ (each equality test can be done in constant time) and there are on average $\Theta(\alpha)$ key-value pairs in the list. Therefore, the total amount of time to compute the hash function and search a linked list at a bucket is $\Theta(1 + \alpha)$.

**Claim** Under the simple uniform hashing assumption the complexity of searching for a key-value pair that *is* in the hash table is in $\Theta(1 + \alpha)$ on average.

**Proof** We assume that the searched for key is equally likely to belong to any one of the $n$ key-value pairs in the hash table. The problem we have is that the key-value pair can appear at any position in any one of the lists.

Assume that adding a key-value pair to a list occurs at the end of the list. When adding the:

* first key-value pair the pair is added to a list whose average length is $0 / m$
    * after adding the pair, the average length of its list is $1 + 0 / m$
* second key-value pair the pair is added to a list whose average length is $1 / m$
    * after adding the pair, the average length of its list is $1 + 0 / m$
* third key-value pair the pair is added to a list whose average length is $2 / m$
    * after adding the pair, the average length of its list is $2 + 0 / m$
* $i$th key-value pair the pair is added to a list whose average length is $(i - 1) / m$
    * after adding the pair, the average length of its list is $1 + (i - 1) / m$

The average length of a list after adding a key-value pair to the list is equal to the number of elements that we have examine to find the key-value pair. The average number of elements that we have to examine over all $n$ keys is therefore equal to:

$$
\begin{align}
\frac{1}{n} \sum_{i=1}^{n}\left(1 + \frac{i - 1}{m} \right) 
& = 1 + \frac{1}{nm}\sum_{i=1}^{n}(i - 1) \\
& = 1 + \left(\frac{1}{nm} \right) \left(\frac{(n-1)n}{2} \right) \\
& = 1 + \frac{\alpha}{2} - \frac{1}{2m}
\end{align}
$$

and the average complexity of searching the list containing the key is $\Theta \left(1 + \frac{\alpha}{2} - \frac{1}{2m}\right)$. The complexity of computing the hash function for the key is in $\Theta(1)$ and the total complexity of searching for a key-value pair that is in the hash table is

$$
\Theta(1) + \Theta \left(1 + \frac{\alpha}{2} - \frac{1}{2m}\right) 
= \Theta \left(2 + \frac{\alpha}{2} - \frac{1}{2m}\right)$$

which is an element of $\Theta(1 + \alpha)$.

If the number of buckets $m$ in the hash table grows in proportion to $n$ then we know that there is some value of $c$ where $n < cm$ is true (i.e., $n$ is an element of $O(m)$) which implies that $\alpha$ is a constant for large values of $n$. Therefore, searching for a key-value pair in a hash table is on average in $O(1)$.

After finding a key-value pair, deleting the key-value pair can be done in $O(1)$ time; therefore, removing a key-value pair is on average in $O(1)$.

If the linked lists maintain a reference to their tail nodes, then adding adding an element to the end of the list takes $O(1)$ time; alternatively, we can add the element at the front of the list. In either case, adding a key-value pair is in $O(1)$ in the worst-case.

**Exercise 1** It is quite easy to create a hash table that uses chaining with linked lists if you have a linked list implementation. Try to do so, perhaps starting by modifying our existing hash table implementation.

**Exercise 2** To preserve the $O(1)$ search complexity, we need to ensure that $m$ grows in proportion with $n$. To do so, we need to re-size the hash table array when the load factor exceeds a certain threshold value. What steps are required when the hash table array is resized?


### Chaining with other data structures

Other data structures can be used instead of a linked list when chaining in a hash table. The obvious choice is to use a balanced binary search tree such as a red-black tree which guarantees $O(\log n)$ complexity for add, remove, and search in the worst case where all keys hash to the same bucket.

One disadvantage of using a binary search tree is that it is usually faster to search a list when the number of elements is small. A second disadvantage of a using a binary search tree is that tree nodes require about two times the amount of memory compared to a linked list node.

The Java standard library class `java.util.HashMap` is a hash table implementation that uses a hybrid approach. A bucket behaves like a linked-node structure when the number of key-value pairs in the bucket is below a threshold value. Above a threshold value, the bucket is transformed into a red-black tree. 